In [18]:
import os
import shutil
import csv
import json
import requests

In [16]:
ISIC2019CSVFile = 'C:\Temp\ISIC_2019_Training_GroundTruth.csv'
ISIC2020CSVFile = 'C:\Temp\ISIC_2020_Training_GroundTruth.csv'

idList = []

with open(ISIC2019CSVFile, 'r') as csv2019:
    reader = csv.reader(csv2019)
    rowCount = 0
    for row in reader:
        if rowCount != 0:
            idList.append(row[0])
        rowCount += 1

print(len(idList))

with open(ISIC2020CSVFile, 'r') as csv2020:
    reader = csv.reader(csv2020)
    rowCount = 0
    for row in reader:
        if rowCount != 0:
            idList.append(row[0])
        rowCount += 1

print(len(idList))


25331
58457


In [17]:
baseAPIstring = 'https://isic-archive.com/api/v1/image'

In [25]:
metadataDirectory = 'C:\Temp\MetaDataFiles'

requestCounter = 0
for item in idList:
    getIDFromName = baseAPIstring + '?name=' + item[0:12]
    responseIDFromName = json.loads(requests.get(getIDFromName).content)

    getMetaDataFromID = baseAPIstring + '/' + responseIDFromName[0]['_id']
    responseMetaDataFromID = json.loads(requests.get(getMetaDataFromID).content)

    fileISICMetadata = metadataDirectory + '\ISIC_Metadata_' + responseIDFromName[0]['_id'] + '.json'
    with open(fileISICMetadata, 'w') as metadataFile:
        json.dump(responseMetaDataFromID, metadataFile, ensure_ascii=False, indent=4)

    requestCounter += 1
    if (requestCounter % 100 == 0):
        print('Total Requests: ' + str(requestCounter))

print('Total Requests: ' + str(requestCounter))

Total Requests: 100
Total Requests: 200
Total Requests: 300
Total Requests: 400
Total Requests: 500
Total Requests: 600
Total Requests: 700
Total Requests: 800
Total Requests: 900
Total Requests: 1000
Total Requests: 1100
Total Requests: 1200
Total Requests: 1300
Total Requests: 1400
Total Requests: 1500
Total Requests: 1600
Total Requests: 1700
Total Requests: 1800
Total Requests: 1900
Total Requests: 2000
Total Requests: 2100
Total Requests: 2200
Total Requests: 2300
Total Requests: 2400
Total Requests: 2500
Total Requests: 2600
Total Requests: 2700
Total Requests: 2800
Total Requests: 2900
Total Requests: 3000
Total Requests: 3100
Total Requests: 3200
Total Requests: 3300
Total Requests: 3400
Total Requests: 3500
Total Requests: 3600
Total Requests: 3700
Total Requests: 3800
Total Requests: 3900
Total Requests: 4000
Total Requests: 4100
Total Requests: 4200
Total Requests: 4300
Total Requests: 4400
Total Requests: 4500
Total Requests: 4600
Total Requests: 4700
Total Requests: 4800
T

In [3]:
metadataDirectory = 'C:\Temp\MetaDataFiles'

allMetadata = []
for fileJSON in os.listdir(metadataDirectory):
    metadataFileName = metadataDirectory + '\\' + os.fsdecode(fileJSON)
    with open(metadataFileName, 'r') as jsonLoad:
        data = json.load(jsonLoad)
        allMetadata.append(data)
print(len(allMetadata))

58457


In [14]:
benignList = []
malignantList = []
otherList = []
allList = []
for item in allMetadata:
    if 'meta' in item:
        if 'clinical' in item['meta']:
            if 'benign_malignant' in item['meta']['clinical']:
                if item['meta']['clinical']['benign_malignant'] == 'benign':
                    benignList.append(item)
                elif item['meta']['clinical']['benign_malignant'] == 'malignant':
                    malignantList.append(item)
                else:
                    otherList.append(item)
            else:
                otherList.append(item)
        else:
            otherList.append(item)
    else:
        otherList.append(item)
print('Allmetadata: ' + str(len(allMetadata)))
print('Benign: ' + str(len(benignList)))
print('Malignant: ' + str(len(malignantList)))
print('Other: ' + str(len(otherList)))
print('Sum: ' + str(len(benignList) + len(malignantList) + len(otherList)))


Allmetadata: 58457
Benign: 37358
Malignant: 5095
Other: 16004
Sum: 58457


In [15]:
malignantMelanomaList = []
malignantOtherList = []
for item in malignantList:
    if item['meta']['clinical']['diagnosis'] == 'melanoma':
        malignantMelanomaList.append(item)
    else:
        malignantOtherList.append(item)
print('Malignant Melanoma: ' + str(len(malignantMelanomaList)))
print('Malignant Other: ' + str(len(malignantOtherList)))


Malignant Melanoma: 5094
Malignant Other: 1


In [24]:
sourceImageDir = 'C:\Temp\ISIC_Images'
destBenignDir = 'C:\Temp\ISIC_Images\BENIGN'
destMalignantMelanomaDir = 'C:\Temp\ISIC_Images\MALIGNANT_MELANOMA'

for item in malignantMelanomaList:
    sourceFileName = sourceImageDir + '\\' + item['name'] + '.jpg'
    destFileName = destMalignantMelanomaDir + '\\' + item['name'] + '.jpg'
    if os.path.exists(sourceFileName):
        shutil.move(sourceFileName, destFileName)

for item in benignList:
    sourceFileName = sourceImageDir + '\\' + item['name'] + '.jpg'
    destFileName = destBenignDir + '\\' + item['name'] + '.jpg'
    if os.path.exists(sourceFileName):
        shutil.move(sourceFileName, destFileName)

print('Malignant Melanoma Images: ' + str(len(os.listdir(destMalignantMelanomaDir))))
print('Benign Images: ' + str(len(os.listdir(destBenignDir))))
print('Excluded Images: ' + str(len(os.listdir(sourceImageDir))))

Malignant Melanoma Images: 4720
Benign Images: 35666
Excluded Images: 18077
